## TCNAE

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.metrics import f1_score, make_scorer, mean_squared_error
import tensorflow as tf
import pandas as pd
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D
from tensorflow.keras.layers import  AveragePooling1D, UpSampling1D
import os
import numpy as np
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, Reshape,Attention, GlobalMaxPooling1D
import random
import pickle
import statistics
from tcn import TCN
import ast
def TCNAEModel(lookback=8, ts_dims = 137, layer1=20, layer2=6, encoding_dim=4):
    model = Sequential([
    TCN(input_shape=(lookback, ts_dims), nb_filters=layer1, kernel_size=3, padding='same',activation='relu', return_sequences=True),
    Conv1D(filters=layer2, kernel_size=3, activation='relu', padding='same'),
    Conv1D(filters=encoding_dim, kernel_size=3, activation='relu', padding='same'),
    AveragePooling1D(pool_size=4, strides=None, padding='valid'),
    Activation("linear"),
    UpSampling1D(size=4),
    Conv1D(filters=layer2, kernel_size=3, activation='relu', padding='same'),
    TCN(nb_filters=layer1, kernel_size=3, padding='same',activation='relu',return_sequences=True),
    Dense(ts_dims, activation='sigmoid')
])
    model.compile(loss='mse', optimizer='adam')
    return model
def temporalize(X, lookback=8):

    output_X = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['Inverter'] == file]['Feature Importance'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2']

def getPIDData():
    path_errors = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/errortimestamps.csv'
    path_features = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/featureslist.csv'
    df_errors = pd.read_csv(path_errors)
    df_features = pd.read_csv(path_features)
    total_features = list(df_features[(df_features['True/False'] == 'TRUE') | (df_features['True/False'] == 'NotAvailable')]['Features'])
    tot_feat = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/totfeat.csv')
    tot_feat = list(tot_feat['feat'])
    xtrain_dfs = {}
    xtest_dfs = {}
    ytrain_dfs = {}
    ytest_dfs = {}
    for inverter in range(19):
        inv = 'inv_'+str(inverter)
        tot_feat.append('ErrBits')
        df_inv_0 = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/downsampledata_PandasMeanNew/'+str(inverter)+'.csv')
        df_inv_0['Timestamp'] = pd.to_datetime(df_inv_0['Timestamp'])
        df_inv_0.sort_values(by='Timestamp', inplace=True)
        df_inv_0 = df_inv_0.set_index(df_inv_0['Timestamp'])
        df_inv_0.shape


        format = '%Y-%m-%d %H:%M:%S'
        error_date = datetime.strptime(df_errors[(df_errors['Inverter'] == inverter) & (df_errors['had_failure'] == True)]['failure_time'].values[0][0:19], format)
        start_date = error_date-timedelta(days= 44)
        end_date = error_date+timedelta(days= 1*30)
        split_date = error_date-timedelta(days= 1*30)

        df_inv_0 = df_inv_0[(df_inv_0.index > start_date) & (df_inv_0.index < end_date)]
        df_inv_0 = pd.get_dummies(df_inv_0)

        for feat in tot_feat:
            if feat not in df_inv_0.columns:
                df_inv_0[feat] = 0
        df_inv_0 = df_inv_0[tot_feat]
        df_inv_0 = df_inv_0.dropna()
        df_target = df_inv_0[['ErrBits']]
        tot_feat.remove('ErrBits')
        df_inv_0 = df_inv_0[tot_feat]

        xtrain, xtest = df_inv_0[(df_inv_0.index >= start_date) & (df_inv_0.index< split_date)], df_inv_0[(df_inv_0.index >= split_date) & (df_inv_0.index< end_date)]
        ytrain, ytest = df_target[(df_target.index >= start_date) & (df_target.index< split_date)], df_target[(df_target.index >= split_date) & (df_target.index< end_date)]
        xtrain_dfs[inv] = xtrain
        xtest_dfs[inv] = xtest
        ytrain_dfs[inv] = ytrain
        ytest_dfs[inv] = ytest
    return xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs
def get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs):
    xtrain, xtest = xtrain_dfs[inv], xtest_dfs[inv]
    ytrain, ytest = ytrain_dfs[inv], ytest_dfs[inv]
    X_scaler = MinMaxScaler()
    X_scaler.fit(xtrain)
    xtrain = X_scaler.transform(xtrain)
    xtest = X_scaler.transform(xtest)
    X_scalar = MinMaxScaler()
    X_scalar.fit(xtrain)
    xtrain, xtest = X_scalar.transform(xtrain), X_scalar.transform(xtest)
    return xtrain, xtest, ytrain, ytest

def other_evaluation(xtrain, xtest, ytest, ytrain, model):
    test_enc = model.predict(xtest)
    train_enc = model.predict(xtrain)
    test_mses = np.mean(np.power(flatten(xtest)- flatten(test_enc), 2), axis=1)
    train_mses = np.mean(np.power(flatten(xtrain)- flatten(train_enc), 2), axis=1)
    test_maes = np.mean(flatten(xtest)- flatten(test_enc), axis=1)
    train_maes = np.mean(flatten(xtrain)- flatten(train_enc), axis=1)
    mse_threshold = np.max(train_mses)
    mae_threshold = np.max(train_maes)
    ypred_mse = np.where(test_mses>mse_threshold,1,0)
    ypred_mae = np.where(test_maes>mae_threshold,1,0)
    test_df = pd.DataFrame(ytest)
    test_df['mse_scores'] = test_mses
    test_df['ytrue'] = np.where(test_df['ErrBits']>0, 1, 0)
    test_df['mae_scores'] = test_maes
    test_df['ypred_mse'] = ypred_mse
    test_df['ypred_mae'] = ypred_mae
    test_df = test_df[['ytrue','ypred_mse','ypred_mae','mse_scores', 'mae_scores']]

    train_df = pd.DataFrame(ytrain)
    train_df['mse_scores'] = train_mses
    train_df['ytrue'] = np.where(train_df['ErrBits']>0, 1, 0)
    train_df['mae_scores'] = train_maes
    train_df = train_df[['ytrue','mse_scores', 'mae_scores']]
    return test_df, train_df
def data_unimodel(source_list, xtrain_dfs):
    train_merge = pd.DataFrame()
    for inverter in source_list:
        X = xtrain_dfs[f'inv_{inverter}']
        X_scaler = MinMaxScaler()
        X_scaler.fit(X)
        xtrain = X_scaler.transform(X)
        df_train = pd.DataFrame(xtrain)
        train_merge = pd.concat([train_merge, df_train], axis=0)
    train_merge_df = np.array(train_merge)
    return train_merge_df
def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2]))
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)
source_list = [12, 4, 6, 14, 10, 18, 5, 8, 3, 0, 11, 2]
total_list = [0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 17, 18]

xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs = getPIDData()
for i in range(1,4):
    df_hyp = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/TCNAE_HYP.csv')
    PID_RESULTSPATH_TOTAL_TRAIN = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/train/'
    PID_RESULTSPATH_TOTAL_TEST = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/test/'
    callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
    lookback = 8
    source_dict_results_test =  {}
    source_dict_results_train = {}
    for file in total_list:
        inv = f'inv_{file}'
        xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
        xtrain = temporalize(xtrain)
        xtest = temporalize(xtest)
        ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
        model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file, df_hyp)
        model  = TCNAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
        model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
        test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
        source_dict_results_test[file] = test_df
        source_dict_results_train[file] = train_df
    with open(PID_RESULTSPATH_TOTAL_TEST+'test_STL_TcnAE.pkl', 'wb') as f:
        pickle.dump(source_dict_results_test, f)
    with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_STL_TcnAE.pkl', 'wb') as f:
        pickle.dump(source_dict_results_train, f)
    train_merge_df = data_unimodel(source_list, xtrain_dfs)
    random.shuffle(train_merge_df)
    train_merge_df = temporalize(train_merge_df)

    model = TCNAEModel(layer1=128, layer2=64, encoding_dim=32)
    model.fit(train_merge_df, train_merge_df, epochs=200, batch_size=128, verbose=2,validation_split=0.1, callbacks=callback1)
    lookback = 8
    mtl_dict_results_test =  {}
    mtl_dict_results_train = {}
    for file in total_list:
        inv = 'inv_'+str(file)
        xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
        xtrain = temporalize(xtrain)
        xtest = temporalize(xtest)
        ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
        test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
        mtl_dict_results_test[file] = test_df
        mtl_dict_results_train[file] = train_df
    with open(PID_RESULTSPATH_TOTAL_TEST+'test_MTL_TcnAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_test, f)
    with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_MTL_TcnAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_train, f)


## TCNATTNAE

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.metrics import f1_score, make_scorer, mean_squared_error
import tensorflow as tf
import pandas as pd
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D
from tensorflow.keras.layers import  AveragePooling1D, UpSampling1D
import os
import numpy as np
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, Reshape,Attention, GlobalMaxPooling1D
import random
import pickle
import statistics
from tcn import TCN
import ast
def TCNAttnAEModel(lookback=8, n_features=137, layer1=100, layer2=64, encoding_dim = 4):
    encoder_inputs = Input(shape=(8, n_features))
    encoder = TCN(nb_filters=layer1, kernel_size=3, padding='same',activation='relu', return_sequences=True)(encoder_inputs)
    encoder = Conv1D(filters=layer2, kernel_size=3, activation='relu', padding='same')(encoder)
    encoder = Conv1D(filters=encoding_dim, kernel_size=3, activation='relu', padding='same')(encoder)
    attention = Attention()([encoder, encoder])
    attention = AveragePooling1D(pool_size=4, strides=None, padding='valid')(attention)
    decoder = UpSampling1D(size=4)(attention)
    decoder = Conv1D(filters=layer2, kernel_size=3, activation='relu', padding='same')(decoder)
    decoder = TCN(nb_filters=layer1, kernel_size=3, padding='same',activation='relu',return_sequences=True)(decoder)
    decoder_outputs = Dense(n_features, activation='sigmoid')(decoder)
    autoencoder = Model(encoder_inputs, decoder_outputs)
    autoencoder.compile(loss='mse', optimizer='adam')    
    return autoencoder
def temporalize(X, lookback=8):

    output_X = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['Inverter'] == file]['Feature Importance'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] 

def getPIDData():
    path_errors = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/errortimestamps.csv'
    path_features = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/featureslist.csv'
    df_errors = pd.read_csv(path_errors)
    df_features = pd.read_csv(path_features)
    total_features = list(df_features[(df_features['True/False'] == 'TRUE') | (df_features['True/False'] == 'NotAvailable')]['Features'])
    tot_feat = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/totfeat.csv')
    tot_feat = list(tot_feat['feat'])
    xtrain_dfs = {}
    xtest_dfs = {}
    ytrain_dfs = {}
    ytest_dfs = {}
    for inverter in range(19):
        inv = 'inv_'+str(inverter)
        tot_feat.append('ErrBits')
        df_inv_0 = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/downsampledata_PandasMeanNew/'+str(inverter)+'.csv')
        df_inv_0['Timestamp'] = pd.to_datetime(df_inv_0['Timestamp'])
        df_inv_0.sort_values(by='Timestamp', inplace=True)
        df_inv_0 = df_inv_0.set_index(df_inv_0['Timestamp'])
        df_inv_0.shape


        format = '%Y-%m-%d %H:%M:%S'
        error_date = datetime.strptime(df_errors[(df_errors['Inverter'] == inverter) & (df_errors['had_failure'] == True)]['failure_time'].values[0][0:19], format)
        start_date = error_date-timedelta(days= 44)
        end_date = error_date+timedelta(days= 1*30)
        split_date = error_date-timedelta(days= 1*30)

        df_inv_0 = df_inv_0[(df_inv_0.index > start_date) & (df_inv_0.index < end_date)]
        df_inv_0 = pd.get_dummies(df_inv_0)

        for feat in tot_feat:
            if feat not in df_inv_0.columns:
                df_inv_0[feat] = 0
        df_inv_0 = df_inv_0[tot_feat]
        df_inv_0 = df_inv_0.dropna()
        df_target = df_inv_0[['ErrBits']]
        tot_feat.remove('ErrBits')
        df_inv_0 = df_inv_0[tot_feat]

        xtrain, xtest = df_inv_0[(df_inv_0.index >= start_date) & (df_inv_0.index< split_date)], df_inv_0[(df_inv_0.index >= split_date) & (df_inv_0.index< end_date)]
        ytrain, ytest = df_target[(df_target.index >= start_date) & (df_target.index< split_date)], df_target[(df_target.index >= split_date) & (df_target.index< end_date)]
        xtrain_dfs[inv] = xtrain
        xtest_dfs[inv] = xtest
        ytrain_dfs[inv] = ytrain
        ytest_dfs[inv] = ytest
    return xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs
def get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs):
    xtrain, xtest = xtrain_dfs[inv], xtest_dfs[inv]
    ytrain, ytest = ytrain_dfs[inv], ytest_dfs[inv]
    X_scaler = MinMaxScaler()
    X_scaler.fit(xtrain)
    xtrain = X_scaler.transform(xtrain)
    xtest = X_scaler.transform(xtest)
    X_scalar = MinMaxScaler()
    X_scalar.fit(xtrain)
    xtrain, xtest = X_scalar.transform(xtrain), X_scalar.transform(xtest)
    return xtrain, xtest, ytrain, ytest

def other_evaluation(xtrain, xtest, ytest, ytrain, model):
    test_enc = model.predict(xtest)
    train_enc = model.predict(xtrain)
    test_mses = np.mean(np.power(flatten(xtest)- flatten(test_enc), 2), axis=1)
    train_mses = np.mean(np.power(flatten(xtrain)- flatten(train_enc), 2), axis=1)
    test_maes = np.mean(flatten(xtest)- flatten(test_enc), axis=1)
    train_maes = np.mean(flatten(xtrain)- flatten(train_enc), axis=1)
    mse_threshold = np.max(train_mses)
    mae_threshold = np.max(train_maes)
    ypred_mse = np.where(test_mses>mse_threshold,1,0)
    ypred_mae = np.where(test_maes>mae_threshold,1,0)
    test_df = pd.DataFrame(ytest)
    test_df['mse_scores'] = test_mses
    test_df['ytrue'] = np.where(test_df['ErrBits']>0, 1, 0)
    test_df['mae_scores'] = test_maes
    test_df['ypred_mse'] = ypred_mse
    test_df['ypred_mae'] = ypred_mae
    test_df = test_df[['ytrue','ypred_mse','ypred_mae','mse_scores', 'mae_scores']]
    
    train_df = pd.DataFrame(ytrain)
    train_df['mse_scores'] = train_mses
    train_df['ytrue'] = np.where(train_df['ErrBits']>0, 1, 0)
    train_df['mae_scores'] = train_maes
    train_df = train_df[['ytrue','mse_scores', 'mae_scores']]
    return test_df, train_df
def data_unimodel(source_list, xtrain_dfs):
    train_merge = pd.DataFrame()
    for inverter in source_list:
        X = xtrain_dfs[f'inv_{inverter}']
        X_scaler = MinMaxScaler()
        X_scaler.fit(X)
        xtrain = X_scaler.transform(X)
        df_train = pd.DataFrame(xtrain)
        train_merge = pd.concat([train_merge, df_train], axis=0)
    train_merge_df = np.array(train_merge)
    return train_merge_df
def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2])) 
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)
source_list = [12, 4, 6, 14, 10, 18, 5, 8, 3, 0, 11, 2]
total_list = [0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 17, 18]

xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs = getPIDData()
for i in range(1,4):
    df_hyp = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/TCNATTNAE_HYP.csv')
    PID_RESULTSPATH_TOTAL_TRAIN = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/train/'
    PID_RESULTSPATH_TOTAL_TEST = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/test/'
    callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
    lookback = 8
    source_dict_results_test =  {}
    source_dict_results_train = {}
    for file in total_list:
        inv = f'inv_{file}'
        xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
        xtrain = temporalize(xtrain)
        xtest = temporalize(xtest)
        ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
        model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file, df_hyp)                                               
        model  = TCNAttnAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
        model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
        test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
        source_dict_results_test[file] = test_df
        source_dict_results_train[file] = train_df
    with open(PID_RESULTSPATH_TOTAL_TEST+'test_STL_TcnAttnAE.pkl', 'wb') as f:
        pickle.dump(source_dict_results_test, f)
    with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_STL_TcnAttnAE.pkl', 'wb') as f:
        pickle.dump(source_dict_results_train, f)
    train_merge_df = data_unimodel(source_list, xtrain_dfs)
    random.shuffle(train_merge_df)
    train_merge_df = temporalize(train_merge_df)

    model = TCNAttnAEModel(layer1=128, layer2=64, encoding_dim=32)
    model.fit(train_merge_df, train_merge_df, epochs=200, batch_size=128, verbose=2,validation_split=0.1, callbacks=callback1)
    lookback = 8
    mtl_dict_results_test =  {}
    mtl_dict_results_train = {}
    for file in total_list:
        inv = 'inv_'+str(file)
        xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
        xtrain = temporalize(xtrain)
        xtest = temporalize(xtest)
        ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
        test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
        mtl_dict_results_test[file] = test_df
        mtl_dict_results_train[file] = train_df
    with open(PID_RESULTSPATH_TOTAL_TEST+'test_MTL_TcnAttnAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_test, f)
    with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_MTL_TcnAttnAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_train, f)

## LSTMAE

In [22]:
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.metrics import f1_score, make_scorer, mean_squared_error
import tensorflow as tf
import pandas as pd
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D
from tensorflow.keras.layers import  AveragePooling1D, UpSampling1D
import os
import numpy as np
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, Reshape,Attention, GlobalMaxPooling1D
import random
import pickle
import statistics
from tcn import TCN
import ast
def LstmAEModel(layer1=20, layer2=6, encoding_dim=4, n_features=137):
    lstm_autoencoder = Sequential()
    lstm_autoencoder.add(LSTM(layer1, activation='relu', input_shape=(8, n_features), return_sequences=True))
    lstm_autoencoder.add(LSTM(layer2, activation='relu', return_sequences=True))
    lstm_autoencoder.add(LSTM(encoding_dim, activation='relu', return_sequences=False))
    lstm_autoencoder.add(RepeatVector(8))
    lstm_autoencoder.add(LSTM(layer2, activation='relu', return_sequences=True))
    lstm_autoencoder.add(LSTM(layer1, activation='relu', return_sequences=True))
    lstm_autoencoder.add(TimeDistributed(Dense(n_features, activation='sigmoid')))

    lstm_autoencoder.compile(loss='mse', optimizer='adam')
    return lstm_autoencoder
def temporalize(X, lookback=8):

    output_X = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['Inverter'] == file]['Feature Importance'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2']

def getPIDData():
    path_errors = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/errortimestamps.csv'
    path_features = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/featureslist.csv'
    df_errors = pd.read_csv(path_errors)
    df_features = pd.read_csv(path_features)
    total_features = list(df_features[(df_features['True/False'] == 'TRUE') | (df_features['True/False'] == 'NotAvailable')]['Features'])
    tot_feat = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/totfeat.csv')
    tot_feat = list(tot_feat['feat'])
    xtrain_dfs = {}
    xtest_dfs = {}
    ytrain_dfs = {}
    ytest_dfs = {}
    for inverter in range(19):
        inv = 'inv_'+str(inverter)
        tot_feat.append('ErrBits')
        df_inv_0 = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/downsampledata_PandasMeanNew/'+str(inverter)+'.csv')
        df_inv_0['Timestamp'] = pd.to_datetime(df_inv_0['Timestamp'])
        df_inv_0.sort_values(by='Timestamp', inplace=True)
        df_inv_0 = df_inv_0.set_index(df_inv_0['Timestamp'])
        df_inv_0.shape


        format = '%Y-%m-%d %H:%M:%S'
        error_date = datetime.strptime(df_errors[(df_errors['Inverter'] == inverter) & (df_errors['had_failure'] == True)]['failure_time'].values[0][0:19], format)
        start_date = error_date-timedelta(days= 44)
        end_date = error_date+timedelta(days= 1*30)
        split_date = error_date-timedelta(days= 1*30)

        df_inv_0 = df_inv_0[(df_inv_0.index > start_date) & (df_inv_0.index < end_date)]
        df_inv_0 = pd.get_dummies(df_inv_0)

        for feat in tot_feat:
            if feat not in df_inv_0.columns:
                df_inv_0[feat] = 0
        df_inv_0 = df_inv_0[tot_feat]
        df_inv_0 = df_inv_0.dropna()
        df_target = df_inv_0[['ErrBits']]
        tot_feat.remove('ErrBits')
        df_inv_0 = df_inv_0[tot_feat]

        xtrain, xtest = df_inv_0[(df_inv_0.index >= start_date) & (df_inv_0.index< split_date)], df_inv_0[(df_inv_0.index >= split_date) & (df_inv_0.index< end_date)]
        ytrain, ytest = df_target[(df_target.index >= start_date) & (df_target.index< split_date)], df_target[(df_target.index >= split_date) & (df_target.index< end_date)]
        xtrain_dfs[inv] = xtrain
        xtest_dfs[inv] = xtest
        ytrain_dfs[inv] = ytrain
        ytest_dfs[inv] = ytest
    return xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs
def get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs):
    xtrain, xtest = xtrain_dfs[inv], xtest_dfs[inv]
    ytrain, ytest = ytrain_dfs[inv], ytest_dfs[inv]
    X_scaler = MinMaxScaler()
    X_scaler.fit(xtrain)
    xtrain = X_scaler.transform(xtrain)
    xtest = X_scaler.transform(xtest)
    X_scalar = MinMaxScaler()
    X_scalar.fit(xtrain)
    xtrain, xtest = X_scalar.transform(xtrain), X_scalar.transform(xtest)
    return xtrain, xtest, ytrain, ytest

def other_evaluation(xtrain, xtest, ytest, ytrain, model):
    test_enc = model.predict(xtest)
    train_enc = model.predict(xtrain)
    test_mses = np.mean(np.power(flatten(xtest)- flatten(test_enc), 2), axis=1)
    train_mses = np.mean(np.power(flatten(xtrain)- flatten(train_enc), 2), axis=1)
    test_maes = np.mean(flatten(xtest)- flatten(test_enc), axis=1)
    train_maes = np.mean(flatten(xtrain)- flatten(train_enc), axis=1)
    mse_threshold = np.max(train_mses)
    mae_threshold = np.max(train_maes)
    ypred_mse = np.where(test_mses>mse_threshold,1,0)
    ypred_mae = np.where(test_maes>mae_threshold,1,0)
    test_df = pd.DataFrame(ytest)
    test_df['mse_scores'] = test_mses
    test_df['ytrue'] = np.where(test_df['ErrBits']>0, 1, 0)
    test_df['mae_scores'] = test_maes
    test_df['ypred_mse'] = ypred_mse
    test_df['ypred_mae'] = ypred_mae
    test_df = test_df[['ytrue','ypred_mse','ypred_mae','mse_scores', 'mae_scores']]

    train_df = pd.DataFrame(ytrain)
    train_df['mse_scores'] = train_mses
    train_df['ytrue'] = np.where(train_df['ErrBits']>0, 1, 0)
    train_df['mae_scores'] = train_maes
    train_df = train_df[['ytrue','mse_scores', 'mae_scores']]
    return test_df, train_df
def data_unimodel(source_list, xtrain_dfs):
    train_merge = pd.DataFrame()
    for inverter in source_list:
        X = xtrain_dfs[f'inv_{inverter}']
        X_scaler = MinMaxScaler()
        X_scaler.fit(X)
        xtrain = X_scaler.transform(X)
        df_train = pd.DataFrame(xtrain)
        train_merge = pd.concat([train_merge, df_train], axis=0)
    train_merge_df = np.array(train_merge)
    return train_merge_df
def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2]))
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)
source_list = [12, 4, 6, 14, 10, 18, 5, 8, 3, 0, 11, 2]
total_list = [0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 17, 18]

xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs = getPIDData()
for i in range(1,4):
    df_hyp = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/LSTMAE_HYP.csv')
    PID_RESULTSPATH_TOTAL_TRAIN = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/train/'
    PID_RESULTSPATH_TOTAL_TEST = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/test/'
    callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
    lookback = 8
    source_dict_results_test =  {}
    source_dict_results_train = {}
    for file in total_list:
        inv = f'inv_{file}'
        xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
        xtrain = temporalize(xtrain)
        xtest = temporalize(xtest)
        ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
        model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file, df_hyp)
        model  = LstmAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
        model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
        test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
        source_dict_results_test[file] = test_df
        source_dict_results_train[file] = train_df
    with open(PID_RESULTSPATH_TOTAL_TEST+'test_STL_LstmAE.pkl', 'wb') as f:
        pickle.dump(source_dict_results_test, f)
    with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_STL_LstmAE.pkl', 'wb') as f:
        pickle.dump(source_dict_results_train, f)
    train_merge_df = data_unimodel(source_list, xtrain_dfs)
    random.shuffle(train_merge_df)
    train_merge_df = temporalize(train_merge_df)

    model = LstmAEModel(layer1=128, layer2=64, encoding_dim=16)
    model.fit(train_merge_df, train_merge_df, epochs=200, batch_size=128, verbose=2,validation_split=0.1, callbacks=callback1)
    lookback = 8
    mtl_dict_results_test =  {}
    mtl_dict_results_train = {}
    for file in total_list:
        inv = 'inv_'+str(file)
        xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
        xtrain = temporalize(xtrain)
        xtest = temporalize(xtest)
        ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
        test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
        mtl_dict_results_test[file] = test_df
        mtl_dict_results_train[file] = train_df
    with open(PID_RESULTSPATH_TOTAL_TEST+'test_MTL_LstmAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_test, f)
    with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_MTL_LstmAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_train, f)


Epoch 1/100
109/109 - 12s - loss: 0.0493 - val_loss: 0.0558 - 12s/epoch - 111ms/step
Epoch 2/100
109/109 - 4s - loss: 0.0237 - val_loss: 0.0538 - 4s/epoch - 36ms/step
Epoch 3/100
109/109 - 4s - loss: 0.0228 - val_loss: 0.0528 - 4s/epoch - 38ms/step
Epoch 4/100
109/109 - 4s - loss: 0.0223 - val_loss: 0.0519 - 4s/epoch - 37ms/step
Epoch 5/100
109/109 - 4s - loss: 0.0219 - val_loss: 0.0514 - 4s/epoch - 38ms/step
Epoch 6/100
109/109 - 4s - loss: 0.0217 - val_loss: 0.0504 - 4s/epoch - 37ms/step
Epoch 7/100
109/109 - 4s - loss: 0.0215 - val_loss: 0.0498 - 4s/epoch - 38ms/step
Epoch 8/100
109/109 - 4s - loss: 0.0214 - val_loss: 0.0496 - 4s/epoch - 40ms/step
Epoch 9/100
109/109 - 4s - loss: 0.0213 - val_loss: 0.0491 - 4s/epoch - 36ms/step
Epoch 10/100
109/109 - 4s - loss: 0.0212 - val_loss: 0.0489 - 4s/epoch - 39ms/step
Epoch 11/100
109/109 - 4s - loss: 0.0212 - val_loss: 0.0490 - 4s/epoch - 39ms/step
Epoch 12/100
109/109 - 4s - loss: 0.0211 - val_loss: 0.0488 - 4s/epoch - 39ms/step
Epoch 13/1

Epoch 100/100
109/109 - 4s - loss: 0.0139 - val_loss: 0.0378 - 4s/epoch - 37ms/step


## LSTMATTNAE

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.metrics import f1_score, make_scorer, mean_squared_error
import tensorflow as tf
import pandas as pd
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D
from tensorflow.keras.layers import  AveragePooling1D, UpSampling1D
import os
import numpy as np
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, Reshape,Attention, GlobalMaxPooling1D
import random
import pickle
import statistics
from tcn import TCN
import ast
def LstmAttnAEModel(layer1=20, layer2=4, encoding_dim=4, n_features=137):
    encoder_inputs = Input(shape=(8, n_features))
    encoder = LSTM(layer1, activation='relu', return_sequences=True)(encoder_inputs)
    encoder = LSTM(layer2, activation='relu', return_sequences=True)(encoder)
    encoder = LSTM(encoding_dim, activation='relu', return_sequences=False)(encoder)
    encoder_outputs = RepeatVector(8)(encoder)
    attention = Attention()([encoder, encoder_outputs])
    attention = GlobalMaxPooling1D()(attention)
    attention = Reshape((1, encoding_dim))(attention)
    decoder = LSTM(layer2, activation='relu', return_sequences=True)(attention)
    decoder = LSTM(layer1, activation='relu', return_sequences=True)(decoder)
    decoder_outputs = TimeDistributed(Dense(n_features,activation='sigmoid'))(decoder)
    autoencoder = Model(encoder_inputs, decoder_outputs)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder
def temporalize(X, lookback=8):

    output_X = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['Inverter'] == file]['Feature Importance'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] 

def getPIDData():
    path_errors = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/errortimestamps.csv'
    path_features = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/featureslist.csv'
    df_errors = pd.read_csv(path_errors)
    df_features = pd.read_csv(path_features)
    total_features = list(df_features[(df_features['True/False'] == 'TRUE') | (df_features['True/False'] == 'NotAvailable')]['Features'])
    tot_feat = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/totfeat.csv')
    tot_feat = list(tot_feat['feat'])
    xtrain_dfs = {}
    xtest_dfs = {}
    ytrain_dfs = {}
    ytest_dfs = {}
    for inverter in range(19):
        inv = 'inv_'+str(inverter)
        tot_feat.append('ErrBits')
        df_inv_0 = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/downsampledata_PandasMeanNew/'+str(inverter)+'.csv')
        df_inv_0['Timestamp'] = pd.to_datetime(df_inv_0['Timestamp'])
        df_inv_0.sort_values(by='Timestamp', inplace=True)
        df_inv_0 = df_inv_0.set_index(df_inv_0['Timestamp'])
        df_inv_0.shape


        format = '%Y-%m-%d %H:%M:%S'
        error_date = datetime.strptime(df_errors[(df_errors['Inverter'] == inverter) & (df_errors['had_failure'] == True)]['failure_time'].values[0][0:19], format)
        start_date = error_date-timedelta(days= 44)
        end_date = error_date+timedelta(days= 1*30)
        split_date = error_date-timedelta(days= 1*30)

        df_inv_0 = df_inv_0[(df_inv_0.index > start_date) & (df_inv_0.index < end_date)]
        df_inv_0 = pd.get_dummies(df_inv_0)

        for feat in tot_feat:
            if feat not in df_inv_0.columns:
                df_inv_0[feat] = 0
        df_inv_0 = df_inv_0[tot_feat]
        df_inv_0 = df_inv_0.dropna()
        df_target = df_inv_0[['ErrBits']]
        tot_feat.remove('ErrBits')
        df_inv_0 = df_inv_0[tot_feat]

        xtrain, xtest = df_inv_0[(df_inv_0.index >= start_date) & (df_inv_0.index< split_date)], df_inv_0[(df_inv_0.index >= split_date) & (df_inv_0.index< end_date)]
        ytrain, ytest = df_target[(df_target.index >= start_date) & (df_target.index< split_date)], df_target[(df_target.index >= split_date) & (df_target.index< end_date)]
        xtrain_dfs[inv] = xtrain
        xtest_dfs[inv] = xtest
        ytrain_dfs[inv] = ytrain
        ytest_dfs[inv] = ytest
    return xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs
def get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs):
    xtrain, xtest = xtrain_dfs[inv], xtest_dfs[inv]
    ytrain, ytest = ytrain_dfs[inv], ytest_dfs[inv]
    X_scaler = MinMaxScaler()
    X_scaler.fit(xtrain)
    xtrain = X_scaler.transform(xtrain)
    xtest = X_scaler.transform(xtest)
    X_scalar = MinMaxScaler()
    X_scalar.fit(xtrain)
    xtrain, xtest = X_scalar.transform(xtrain), X_scalar.transform(xtest)
    return xtrain, xtest, ytrain, ytest

def other_evaluation(xtrain, xtest, ytest, ytrain, model):
    test_enc = model.predict(xtest)
    train_enc = model.predict(xtrain)
    test_mses = np.mean(np.power(flatten(xtest)- flatten(test_enc), 2), axis=1)
    train_mses = np.mean(np.power(flatten(xtrain)- flatten(train_enc), 2), axis=1)
    test_maes = np.mean(flatten(xtest)- flatten(test_enc), axis=1)
    train_maes = np.mean(flatten(xtrain)- flatten(train_enc), axis=1)
    mse_threshold = np.max(train_mses)
    mae_threshold = np.max(train_maes)
    ypred_mse = np.where(test_mses>mse_threshold,1,0)
    ypred_mae = np.where(test_maes>mae_threshold,1,0)
    test_df = pd.DataFrame(ytest)
    test_df['mse_scores'] = test_mses
    test_df['ytrue'] = np.where(test_df['ErrBits']>0, 1, 0)
    test_df['mae_scores'] = test_maes
    test_df['ypred_mse'] = ypred_mse
    test_df['ypred_mae'] = ypred_mae
    test_df = test_df[['ytrue','ypred_mse','ypred_mae','mse_scores', 'mae_scores']]
    
    train_df = pd.DataFrame(ytrain)
    train_df['mse_scores'] = train_mses
    train_df['ytrue'] = np.where(train_df['ErrBits']>0, 1, 0)
    train_df['mae_scores'] = train_maes
    train_df = train_df[['ytrue','mse_scores', 'mae_scores']]
    return test_df, train_df
def data_unimodel(source_list, xtrain_dfs):
    train_merge = pd.DataFrame()
    for inverter in source_list:
        X = xtrain_dfs[f'inv_{inverter}']
        X_scaler = MinMaxScaler()
        X_scaler.fit(X)
        xtrain = X_scaler.transform(X)
        df_train = pd.DataFrame(xtrain)
        train_merge = pd.concat([train_merge, df_train], axis=0)
    train_merge_df = np.array(train_merge)
    return train_merge_df
def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2])) 
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)
source_list = [12, 4, 6, 14, 10, 18, 5, 8, 3, 0, 11, 2]
total_list = [0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 17, 18]

xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs = getPIDData()
for i in range(1,4):
    df_hyp = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/LSTMATTNAE_HYP.csv')
    PID_RESULTSPATH_TOTAL_TRAIN = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/train/'
    PID_RESULTSPATH_TOTAL_TEST = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/test/'
    callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
    lookback = 8
    source_dict_results_test =  {}
    source_dict_results_train = {}
    for file in total_list:
        inv = f'inv_{file}'
        xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
        xtrain = temporalize(xtrain)
        xtest = temporalize(xtest)
        ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
        model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file, df_hyp)                                               
        model  = LstmAttnAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
        model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
        test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
        source_dict_results_test[file] = test_df
        source_dict_results_train[file] = train_df
    with open(PID_RESULTSPATH_TOTAL_TEST+'test_STL_LstmAttnAE.pkl', 'wb') as f:
        pickle.dump(source_dict_results_test, f)
    with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_STL_LstmAttnAE.pkl', 'wb') as f:
        pickle.dump(source_dict_results_train, f)
    train_merge_df = data_unimodel(source_list, xtrain_dfs)
    random.shuffle(train_merge_df)
    train_merge_df = temporalize(train_merge_df)

    model = LstmAttnAEModel(layer1=64, layer2=64, encoding_dim=32)
    model.fit(train_merge_df, train_merge_df, epochs=200, batch_size=128, verbose=2,validation_split=0.1, callbacks=callback1)
    lookback = 8
    mtl_dict_results_test =  {}
    mtl_dict_results_train = {}
    for file in total_list:
        inv = 'inv_'+str(file)
        xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
        xtrain = temporalize(xtrain)
        xtest = temporalize(xtest)
        ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
        test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
        mtl_dict_results_test[file] = test_df
        mtl_dict_results_train[file] = train_df
    with open(PID_RESULTSPATH_TOTAL_TEST+'test_MTL_LstmAttnAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_test, f)
    with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_MTL_LstmAttnAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_train, f)